# Final Project

## TRAC2- Transformer Models (BERT) - base uncased - Evaluate variability

The purpose of this notebook is to evaluate the variability in the results of the models for classification task-A and task-B.

These models use the oversampled training data. For more details about the oversampling method refer to the `TRAC2-Data_oversampling.ipynb` notebook.

In [4]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Package imports

In [5]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 14.8 MB/s 
     |████████████████████████████████| 895 kB 49.1 MB/s 
     |████████████████████████████████| 59 kB 9.3 MB/s 
     |████████████████████████████████| 3.3 MB 65.8 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
!pip install datasets

     |████████████████████████████████| 290 kB 15.3 MB/s 
     |████████████████████████████████| 132 kB 56.5 MB/s 
     |████████████████████████████████| 243 kB 69.8 MB/s 
     |████████████████████████████████| 1.1 MB 66.1 MB/s 
     |████████████████████████████████| 160 kB 77.3 MB/s 
     |████████████████████████████████| 271 kB 56.8 MB/s 
     |████████████████████████████████| 192 kB 74.4 MB/s 


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from datasets import load_dataset

from sklearn.preprocessing import label_binarize
from sklearn import metrics

import statistics

## Helper functions

In [ ]:
def from_logits_to_labels(pred, task):
    '''
    Returns labels based on predicted logits on labels [CAG,NAG,OAG] for task A. Task B is binary, and 'GEN' represents 
    the positive class.
    Parameters:
    pred: array with model prediction
    task: either 'A' or 'B'
    '''
    index_a = {0:'CAG', 1:'NAG', 2:'OAG'}
    index_b = {0:'GEN', 1:'NGEN'}
    
    if task == 'A':
        highest_prob_class = np.argmax(pred, axis=1)
        labels = np.vectorize(index_a.get)(highest_prob_class.astype(int))
        
    elif task == 'B':
        highest_prob_class = np.argmax(pred, axis=1)
        labels = np.vectorize(index_b.get)(highest_prob_class.astype(int))
    else:
        labels = []
        
    return labels  

In [ ]:
def to_one_hot_labels(string_labels):
    '''
    Returns one-hot encoded labels from a multi-class label vector e.g. ['cat', 'dog', 'dog', 'lion', 'cat', ...] 
    Parameters:
    string_labels: 
    '''
    labels = pd.get_dummies(string_labels)
    labels = labels.to_numpy()
    
    return labels

In [ ]:
# this is modified to get the prediction as parameter
# to avoid predicting again since inference takes time
def confusion_matrix_plot(pred_labels, true_labels, task, normalize=None):
    '''
    Returns a confusion matrix with a nice format.
    Parameters:
    pred_labels: predicted labels
    true_labels: true labels 
    task: 'A' or 'B'
    normalize: if want to normalize the confusion matrix normalize='true'
    '''
    
    # Create a confusion matrix
    cm = metrics.confusion_matrix(true_labels, pred_labels, normalize=normalize)
    cm = np.around(cm, 2)

    # Plot the confusion matrix
    if task == 'A':
        axis_labels = ['CAG', 'NAG', 'OAG']
    elif task == 'B':
        axis_labels = ['GEN', 'NGEN']

    fig, ax = plt.subplots(figsize=(4,4))
    im = ax.imshow(cm, cmap="Blues")

    # Create the ticks and labels
    ax.set_xticks(np.arange(len(axis_labels)))
    ax.set_yticks(np.arange(len(axis_labels)))
    ax.set_xticklabels(axis_labels)
    ax.set_yticklabels(axis_labels)

    # Axis titles
    plt.ylabel('True label', size=12)
    plt.xlabel('Predicted label', size=12)

    # Loop over data dimensions and create text annotations.
    for i in range(len(axis_labels)):
        for j in range(len(axis_labels)):
            text = ax.text(j, i, cm[i, j],ha="center", va="center", color="dimgrey", size=12)
    
    ax.set_title("Confusion Matrix", size=16, weight="bold")
    fig.tight_layout()
    plt.show()


In [ ]:
def loss_accuracy_plots(training_history, xrange, task):
    '''
    Returns plots for loss and accuracy during the training process of a NN.
    Parameters:
    training_history: object that stores the training history of the NN (from model.fit(...))
    xrange: range in x axis
    task: string used for the title in the plot
    '''
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,6))
    
    # loss plot
    ax1.plot(training_history.history['loss'], color='black')
    ax1.plot(training_history.history['val_loss'], color='blue')
    ax1.set_title('Training and validation loss Sub-Task ' + task)
    ax1.legend(['training', 'development'])
    ax1.grid(which='both')
    ax1.set_xticks(np.arange(0, xrange, 2))
    
    # accuracy plot
    ax2.plot(training_history.history['categorical_accuracy'], color='black')
    ax2.plot(training_history.history['val_categorical_accuracy'], color='blue')
    ax2.set_title('Training and validation acccuracy Sub_Task ' + task)
    ax2.legend(['training', 'development'])
    ax2.grid(which='both')
    ax2.set_xticks(np.arange(0, xrange, 2))
    plt.show()
    

## Load data
Load training, development and test datasets.

In [8]:
# Load labels using pandas dataframes

train_labels_a = pd.read_csv('drive/MyDrive/w266/release-files/eng/trac2_eng_train_oversampled_task_A.csv')['Sub-task A']
train_labels_b = pd.read_csv('drive/MyDrive/w266/release-files/eng/trac2_eng_train_oversampled_task_B.csv')['Sub-task B']
dev_labels_a = pd.read_csv('drive/MyDrive/w266/release-files/eng/trac2_eng_dev.csv')['Sub-task A']
dev_labels_b = pd.read_csv('drive/MyDrive/w266/release-files/eng/trac2_eng_dev.csv')['Sub-task B']
test_labels_a = pd.read_csv('drive/MyDrive/w266/release-files/gold/trac2_eng_gold_a.csv')['Sub-task A']
test_labels_b = pd.read_csv('drive/MyDrive/w266/release-files/gold/trac2_eng_gold_b.csv')['Sub-task B']

In [ ]:
# Load text data using Hugging Face datasets
# need to use the split argument even though we are not splitting. If not, data is loaded as DatasetDict
# to load as dataset need to include the split parameter
train_dataset_a = load_dataset('csv', data_files='drive/MyDrive/w266/release-files/eng/trac2_eng_train_oversampled_task_A.csv', split = 'train[:10125]')
train_dataset_b = load_dataset('csv', data_files='drive/MyDrive/w266/release-files/eng/trac2_eng_train_oversampled_task_B.csv', split = 'train[:7908]')
dev_dataset = load_dataset('csv', data_files='drive/MyDrive/w266/release-files/eng/trac2_eng_dev.csv', split = 'train[:1066]')
test_dataset = load_dataset('csv', data_files='drive/MyDrive/w266/release-files/test/trac2_eng_test.csv', split = 'train[:1200]')

Using custom data configuration default-e389f4ee5eb22a64


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e389f4ee5eb22a64/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


Using custom data configuration default-9b2f39c75ffd1fc0


...

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a64066573b62035d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


Using custom data configuration default-d1136119f167d116


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d1136119f167d116/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


## Encode labels

In [ ]:
# encode labels Task A- [CAG,NAG,OAG]
train_labels_a_enc = to_one_hot_labels(train_labels_a)
dev_labels_a_enc = to_one_hot_labels(dev_labels_a)
test_labels_a_enc = to_one_hot_labels(test_labels_a)


In [ ]:
# encode labels Task B- [GEN, NGEN]
train_labels_b_enc = to_one_hot_labels(train_labels_b)
dev_labels_b_enc = to_one_hot_labels(dev_labels_b)
test_labels_b_enc = to_one_hot_labels(test_labels_b)

## Prepare TensorFlow datasets for BERT

In [ ]:
# remove columns to leave only the column with the posts. Column 'Text'
train_dataset_a = train_dataset_a.remove_columns(['ID', 'Sub-task B', 'Sub-task A'])
train_dataset_b = train_dataset_b.remove_columns(['ID', 'Sub-task B', 'Sub-task A'])
dev_dataset = dev_dataset.remove_columns(['ID', 'Sub-task A', 'Sub-task B'])
test_dataset = test_dataset.remove_columns('ID')

In [ ]:
# define a BERT tokenizer
# use the bert-based-uncased tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
# tokenize the train, development and test data
# Use a max sequence of 150 tokens. Based on EDA this is enough for majority of posts

train_dataset_tok_a = train_dataset_a.map(lambda x: tokenizer(x['Text'], truncation=True, padding=True, max_length=150), batched=True)
train_dataset_tok_b = train_dataset_b.map(lambda x: tokenizer(x['Text'], truncation=True, padding=True, max_length=150), batched=True)
dev_dataset_tok = dev_dataset.map(lambda x: tokenizer(x['Text'], truncation=True, padding=True, max_length=150), batched=True)
test_dataset_tok = test_dataset.map(lambda x: tokenizer(x['Text'], truncation=True, padding=True, max_length=150), batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
# now we can remove the column with the original post from the dataset. We are going to use the result of tokenization for modeling
train_dataset_tok_a = train_dataset_tok_a.remove_columns(['Text']).with_format('tensorflow')
train_dataset_tok_b = train_dataset_tok_b.remove_columns(['Text']).with_format('tensorflow')
dev_dataset_tok = dev_dataset_tok.remove_columns(['Text']).with_format('tensorflow')
test_dataset_tok = test_dataset_tok.remove_columns(['Text']).with_format('tensorflow')

In [ ]:
# extract features from tokenizer output: 'input_ids', 'token_type_ids', 'attention_mask'
train_features_a = {x: train_dataset_tok_a[x] for x in tokenizer.model_input_names}
train_features_b = {x: train_dataset_tok_b[x] for x in tokenizer.model_input_names}
dev_features = {x: dev_dataset_tok[x] for x in tokenizer.model_input_names}
test_features = {x: test_dataset_tok[x] for x in tokenizer.model_input_names}

In [ ]:
# batch data

batch_size = 16
buffer_a = len(train_dataset_tok_a)
buffer_b = len(train_dataset_tok_b)

# Task A
train_tf_dataset_a = tf.data.Dataset.from_tensor_slices((train_features_a, train_labels_a_enc)).shuffle(buffer_a).batch(batch_size)
dev_tf_dataset_a = tf.data.Dataset.from_tensor_slices((dev_features, dev_labels_a_enc)).batch(batch_size)
test_tf_dataset_a = tf.data.Dataset.from_tensor_slices((test_features, test_labels_a_enc)).batch(batch_size)

# Task B
train_tf_dataset_b = tf.data.Dataset.from_tensor_slices((train_features_b, train_labels_b_enc)).shuffle(buffer_b).batch(batch_size)
dev_tf_dataset_b = tf.data.Dataset.from_tensor_slices((dev_features, dev_labels_b_enc)).batch(batch_size)
test_tf_dataset_b = tf.data.Dataset.from_tensor_slices((test_features, test_labels_b_enc)).batch(batch_size)

## Model Task A

In [ ]:
# initialize lists to keep statistics of all runs
f1_NAG = []
f1_CAG = []
f1_OAG = []
f1_macro = []
f1_weighted = []
accuracy = []

# run 15 times the model to get an idea of variability
for i in range(5):

  # delete model if exists
  try:
    del BERT_model_A
  except:
    pass
  
  # define the model. Task A is a classification task with 3 labels
  BERT_model_A = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

  # compile model
  BERT_model_A.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                       loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                       metrics=tf.metrics.CategoricalAccuracy()
                       )
  # fit model
  training_history = BERT_model_A.fit(train_tf_dataset_a, validation_data=dev_tf_dataset_a, epochs=2)

  print(f'---------------------------Iteration {i} ---------------------------\n')
  # Evaluate model on TEST data
  # predict using model. Returns logits
  pred_labels_test = BERT_model_A.predict(test_features)[0]
  # convert logits lo labels
  pred_labels_test = from_logits_to_labels(pred_labels_test, 'A')

  # get f1-score for all classes, macro and weighted
  x = metrics.classification_report(test_labels_a, pred_labels_test, digits=3, output_dict=True)
  # append values to keep scores
  f1_NAG.append(x['NAG']['f1-score'])
  f1_CAG.append(x['CAG']['f1-score'])
  f1_OAG.append(x['OAG']['f1-score'])
  f1_macro.append(x['macro avg']['f1-score'])
  f1_weighted.append(x['weighted avg']['f1-score'])
  accuracy.append(x['accuracy'])

# calculate mean
f1_NAG_mean = round(statistics.mean(f1_NAG), 3)
f1_CAG_mean = round(statistics.mean(f1_CAG), 3)
f1_OAG_mean = round(statistics.mean(f1_OAG), 3)
f1_macro_mean = round(statistics.mean(f1_macro), 3)
f1_weighted_mean = round(statistics.mean(f1_weighted), 3)
accuracy_mean = round(statistics.mean(accuracy), 3)

# calculate standard deviation
f1_NAG_std = round(statistics.stdev(f1_NAG), 3)
f1_CAG_std = round(statistics.stdev(f1_CAG), 3)
f1_OAG_std = round(statistics.stdev(f1_OAG), 3)
f1_macro_std = round(statistics.stdev(f1_macro), 3)
f1_weighted_std = round(statistics.stdev(f1_weighted), 3)
accuracy_std = round(statistics.stdev(accuracy), 3)

print('Class NAG')
print(f'Mean f1-score = {f1_NAG_mean}')
print(f'Standard deviation f1-score = {f1_NAG_std}\n')

print('Class CAG')
print(f'Mean f1-score = {f1_CAG_mean}')
print(f'Standard deviation f1-score = {f1_CAG_std}\n')

print('Class OAG')
print(f'Mean f1-score = {f1_OAG_mean}')
print(f'Standard deviation f1-score = {f1_OAG_std}\n')

print('Class Macro')
print(f'Mean f1-score = {f1_macro_mean}')
print(f'Standard deviation f1-score = {f1_macro_std}\n')

print('Class Weighted')
print(f'Mean f1-score = {f1_weighted_mean}')
print(f'Standard deviation f1-score = {f1_weighted_std}\n')

print('Accuracy')
print(f'Mean = {accuracy_mean}')
print(f'Standard deviation = {accuracy_std}\n')

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
633/633 [==============================] - 238s 350ms/step - loss: 0.4442 - categorical_accuracy: 0.8208 - val_loss: 0.6576 - val_categorical_accuracy: 0.7946
Epoch 2/2
633/633 [==============================] - 219s 347ms/step - loss: 0.0971 - categorical_accuracy: 0.9692 - val_loss: 0.8281 - val_categorical_accuracy: 0.8086
---------------------------Iteration 0 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
633/633 [==============================] - 235s 350ms/step - loss: 0.4301 - categorical_accuracy: 0.8249 - val_loss: 0.6475 - val_categorical_accuracy: 0.8002
Epoch 2/2
633/633 [==============================] - 219s 347ms/step - loss: 0.0880 - categorical_accuracy: 0.9732 - val_loss: 0.7408 - val_categorical_accuracy: 0.8077
---------------------------Iteration 1 ---------------------------



...

Epoch 1/2
633/633 [==============================] - 235s 350ms/step - loss: 0.4275 - categorical_accuracy: 0.8269 - val_loss: 0.7139 - val_categorical_accuracy: 0.7645
Epoch 2/2
633/633 [==============================] - 219s 347ms/step - loss: 0.0967 - categorical_accuracy: 0.9706 - val_loss: 0.8454 - val_categorical_accuracy: 0.8077
---------------------------Iteration 2 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
633/633 [==============================] - 235s 350ms/step - loss: 0.4430 - categorical_accuracy: 0.8213 - val_loss: 0.6831 - val_categorical_accuracy: 0.7861
Epoch 2/2
633/633 [==============================] - 219s 346ms/step - loss: 0.0936 - categorical_accuracy: 0.9719 - val_loss: 1.0566 - val_categorical_accuracy: 0.7683
---------------------------Iteration 3 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
633/633 [==============================] - 235s 350ms/step - loss: 0.4611 - categorical_accuracy: 0.8137 - val_loss: 0.6852 - val_categorical_accuracy: 0.7758
Epoch 2/2
633/633 [==============================] - 219s 346ms/step - loss: 0.1017 - categorical_accuracy: 0.9676 - val_loss: 0.8120 - val_categorical_accuracy: 0.7861
---------------------------Iteration 4 ---------------------------

Class NAG
Mean f1-score = 0.838
Standard deviation f1-score = 0.012

Class CAG
Mean f1-score = 0.342
Standard deviation f1-score = 0.079

Class OAG
Mean f1-score = 0.631
Standard deviation f1-score = 0.064

Class Macro
Mean f1-score = 0.604
Standard deviation f1-score = 0.027

Class Weighted
Mean f1-score = 0.696
Standard deviation f1-score = 0.02

Accuracy
Mean = 0.722
Standard deviation = 0.021



## Model Task B

In [ ]:
# initialize lists to keep statistics of all runs
f1_NGEN = []
f1_GEN = []
f1_macro_b = []
f1_weighted_b = []
accuracy_b = []

for i in range(5):

  # delete model if exists
  try:
    del BERT_model_B
  except:
    pass
  
  # define the model. Task B is a classification task with 2 labels
  BERT_model_B = TFAutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

  # compile model
  BERT_model_B.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
                       loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                       metrics=tf.metrics.CategoricalAccuracy()
                       )
  
  # fit model
  training_history = BERT_model_B.fit(train_tf_dataset_b, validation_data=dev_tf_dataset_b, epochs=3)

  print(f'---------------------------Iteration {i} ---------------------------\n')
  # Evaluate model on TEST data
  # predict using model. Returns logits
  pred_labels_test = BERT_model_B.predict(test_features)[0]

  # convert logits lo labels
  pred_labels_test = from_logits_to_labels(pred_labels_test, 'B')

  # get f1-score for all classes, macro and weighted
  x = metrics.classification_report(test_labels_b, pred_labels_test, digits=3, output_dict=True)
  # append values to keep scores
  f1_NGEN.append(x['NGEN']['f1-score'])
  f1_GEN.append(x['GEN']['f1-score'])
  f1_macro_b.append(x['macro avg']['f1-score'])
  f1_weighted_b.append(x['weighted avg']['f1-score'])
  accuracy_b.append(x['accuracy'])

# calculate mean
f1_NGEN_mean = round(statistics.mean(f1_NGEN), 3)
f1_GEN_mean = round(statistics.mean(f1_GEN), 3)
f1_macro_b_mean = round(statistics.mean(f1_macro_b), 3)
f1_weighted_b_mean = round(statistics.mean(f1_weighted_b), 3)
accuracy_b_mean = round(statistics.mean(accuracy_b), 3)

# calculate standard deviation
f1_NGEN_std = round(statistics.stdev(f1_NGEN), 3)
f1_GEN_std = round(statistics.stdev(f1_GEN), 3)
f1_macro_b_std = round(statistics.stdev(f1_macro_b), 3)
f1_weighted_b_std = round(statistics.stdev(f1_weighted_b), 3)
accuracy_b_std = round(statistics.stdev(accuracy_b), 3)

print('Class NGEN')
print(f'Mean f1-score = {f1_NGEN_mean}')
print(f'Standard deviation f1-score = {f1_NGEN_std}\n')

print('Class GEN')
print(f'Mean f1-score = {f1_GEN_mean}')
print(f'Standard deviation f1-score = {f1_GEN_std}\n')

print('Macro')
print(f'Mean f1-score = {f1_macro_b_mean}')
print(f'Standard deviation f1-score = {f1_macro_b_std}\n')

print('Weighted')
print(f'Mean f1-score = {f1_weighted_b_mean}')
print(f'Standard deviation f1-score = {f1_weighted_b_std}\n')

print('Accuracy')
print(f'Mean = {accuracy_b_mean}')
print(f'Standard deviation = {accuracy_b_std}\n')


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
495/495 [==============================] - 188s 354ms/step - loss: 0.1986 - categorical_accuracy: 0.9203 - val_loss: 0.2887 - val_categorical_accuracy: 0.9128
Epoch 2/3
495/495 [==============================] - 173s 349ms/step - loss: 0.0349 - categorical_accuracy: 0.9910 - val_loss: 0.2990 - val_categorical_accuracy: 0.9428
Epoch 3/3
495/495 [==============================] - 173s 350ms/step - loss: 0.0302 - categorical_accuracy: 0.9917 - val_loss: 0.2965 - val_categorical_accuracy: 0.9390
---------------------------Iteration 0 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
495/495 [==============================] - 188s 354ms/step - loss: 0.1866 - categorical_accuracy: 0.9302 - val_loss: 0.2374 - val_categorical_accuracy: 0.9456
Epoch 2/3
495/495 [==============================] - 173s 349ms/step - loss: 0.0395 - categorical_accuracy: 0.9885 - val_loss: 0.3134 - val_categorical_accuracy: 0.9371
Epoch 3/3
495/495 [==============================] - 173s 349ms/step - loss: 0.0251 - categorical_accuracy: 0.9925 - val_loss: 0.3358 - val_categorical_accuracy: 0.9212
---------------------------Iteration 1 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
495/495 [==============================] - 189s 354ms/step - loss: 0.1986 - categorical_accuracy: 0.9222 - val_loss: 0.2747 - val_categorical_accuracy: 0.9268
Epoch 2/3
495/495 [==============================] - 173s 349ms/step - loss: 0.0490 - categorical_accuracy: 0.9865 - val_loss: 0.2597 - val_categorical_accuracy: 0.9400
Epoch 3/3
495/495 [==============================] - 173s 349ms/step - loss: 0.0201 - categorical_accuracy: 0.9947 - val_loss: 0.3604 - val_categorical_accuracy: 0.9437
---------------------------Iteration 2 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
495/495 [==============================] - 189s 354ms/step - loss: 0.1964 - categorical_accuracy: 0.9241 - val_loss: 0.3119 - val_categorical_accuracy: 0.9325
Epoch 2/3
495/495 [==============================] - 173s 349ms/step - loss: 0.0531 - categorical_accuracy: 0.9850 - val_loss: 0.2941 - val_categorical_accuracy: 0.9390
Epoch 3/3
495/495 [==============================] - 173s 349ms/step - loss: 0.0265 - categorical_accuracy: 0.9920 - val_loss: 0.3419 - val_categorical_accuracy: 0.9428
---------------------------Iteration 3 ---------------------------



All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
495/495 [==============================] - 190s 354ms/step - loss: 0.1755 - categorical_accuracy: 0.9264 - val_loss: 0.2586 - val_categorical_accuracy: 0.9465
Epoch 2/3
495/495 [==============================] - 173s 350ms/step - loss: 0.0352 - categorical_accuracy: 0.9893 - val_loss: 0.3036 - val_categorical_accuracy: 0.9306
Epoch 3/3
495/495 [==============================] - 173s 350ms/step - loss: 0.0172 - categorical_accuracy: 0.9947 - val_loss: 0.4387 - val_categorical_accuracy: 0.8809
---------------------------Iteration 4 ---------------------------

Class NGEN
Mean f1-score = 0.892
Standard deviation f1-score = 0.034

Class GEN
Mean f1-score = 0.505
Standard deviation f1-score = 0.016

Macro
Mean f1-score = 0.698
Standard deviation f1-score = 0.017

Weighted
Mean f1-score = 0.836
Standard deviation f1-score = 0.029

Accuracy
Mean = 0.825
Standard deviation = 0.044



## References

- Pre-processing data: https://huggingface.co/transformers/preprocessing.html

- Fine-tunning a pre-trained model: https://huggingface.co/transformers/training.html

- BERT: https://huggingface.co/transformers/model_doc/bert.html
